In [3]:
import tensorflow as tf

# Low Level API

- Tensorflow program (tf.Graph)
- Tensorflow runtime (tf.Session)

## Tensor Values
- array와 같은 형식 값
- rank : 차원 수
- shape : array 모양

In [7]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0)
total = a + b

In [8]:
print(a, b, total)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32) Tensor("add_1:0", shape=(), dtype=float32)


## Graph
- tf.Operation: graph node, 텐서를 생성하거나 소비하는 계산
- tf.Tensor: graph edge, 결과 값 (tf.Tensors)
  - Tensor는 실행될 결과를 표현한다.
  - 각 tensor는 name을 가지고 있다
  - 값을 가지고 있지 않는다.

## TensorBoard
- graph를 visualization해준다.

## Session
- tensor evaluate하기 위한 Session
- tf.Session().run() : 해당 tensor를 evaluate해준다.
- run할 때, tensor는 오직 하나의 값만 가진다.
- 종종 tf.Tensor 대신에 tf.operator을 return한다. 그경우, None 노출

In [10]:
sess = tf.Session()
print(sess.run(total))
print(sess.run({'ab':(a, b), 'total': total}))

7.0
{'ab': (3.0, 4.0), 'total': 7.0}


### Feed
- placeholders : 값을 받을 것을 약속한다.
  - x=tf.placeholder(tf.float32)
  - session.run()에 feed_dict을 이용해 결과 값을 받는다.
  - feed_dict은 Tensor에 overwrite가 가능(placeholder와 차이는 없을 때 오류 출력)

In [11]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [13]:
print(sess.run(z, feed_dict={x:3, y:4}))

7.0


## Datasets
- tf.data 모델에 데이터를 줄 때 더 많이 사용하는 모듈
- Tensor를 tf.data.Iterator로 변경
- tf.data.Iterator,get_next로 사용
- slices.make_one_shot_iterator()로 Iterator 생성
  - 마지막 값 출력시 Error 출력
  - 단 한번만 출력
- slices.make_initializable_iterator()
  - 사용전에 sess.run(iterator.initializer) 실행

In [34]:
my_data = [[0, 1], [2, 3], [4, 5], [6, 7]]

In [35]:
slices = tf.data.Dataset.from_tensor_slices(my_data)

In [36]:
next_time = slices.make_one_shot_iterator().get_next()

In [37]:
while True:
    try:
        print(sess.run(next_time))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


## Layers
- trianable parameter를 추가하는 방법
- variable, operator로 이루어짐
- placeholder 넣을 때 반드시 shape를 설정이 필요
- tf.global_variables_initializer()로 초기화 필요 (return이 operator)

In [40]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

In [41]:
init = tf.global_variables_initializer()
sess.run(init)

In [43]:
sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]})

array([[1.480969],
       [3.826353]], dtype=float32)

### shortcut Layers
- shortcut function : create와 run을 동시에 가능하다.
- tf.layers.Layer 사용이 불가
- introspection과 디버깅이 힘듬, layer 재사용이 불가능

In [45]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

Instructions for updating:
Use keras.layers.dense instead.
[[-1.1721786]
 [-3.981616 ]]


## Training

In [46]:
# Define data
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

In [47]:
# Define model
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [48]:
# session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.30081904]
 [0.6016381 ]
 [0.9024571 ]
 [1.2032762 ]]


### Loss
- tf.losses : loss function 제공

In [51]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

7.186881


### Training
- tf.train.Optimizer : optimizer 수행 
  - optimizer.minimize() : minimizer할 loss 입력 (return operator)
  - run시 loss를 같이 넣어 loss를 확인

In [53]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [54]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)

7.186881
5.070614
3.60168
2.5819197
1.8738348
1.3820176
1.0402668
0.80264705
0.6372845
0.52206266
0.44163507
0.38535345
0.34582895
0.31793478
0.29811347
0.2838965
0.27357092
0.26594847
0.26020426
0.25576606
0.25223678
0.2493408
0.24688709
0.24474278
0.24281587
0.24104242
0.23937805
0.23779194
0.23626272
0.2347756
0.23332016
0.23188922
0.23047788
0.22908267
0.22770098
0.22633125
0.22497234
0.22362322
0.22228345
0.22095254
0.2196301
0.21831606
0.21701017
0.2157122
0.21442218
0.21313995
0.21186548
0.2105987
0.20933951
0.20808786
0.2068437
0.20560692
0.20437765
0.20315573
0.20194107
0.20073366
0.1995335
0.19834052
0.1971547
0.19597593
0.19480425
0.19363952
0.19248174
0.19133097
0.19018702
0.18904993
0.18791966
0.1867961
0.18567927
0.18456909
0.18346561
0.18236867
0.18127832
0.1801945
0.17911713
0.17804626
0.17698175
0.17592357
0.17487174
0.1738262
0.17278692
0.1717539
0.170727
0.16970623
0.16869158
0.16768304
0.16668049
0.16568391
0.16469331
0.16370864
0.16272986
0.16175693
0.16078982
0.15

In [55]:
print(sess.run(y_pred))

[[-0.63212246]
 [-1.3063067 ]
 [-1.9804908 ]
 [-2.654675  ]]


## Summary
- tf.Graph, tf.Session 두가지 존재
  - tf.Graph = tf.Tensor(Edge) + tf.Operator(Node)
  - tf.Tensor -> tf.Constant, tf.Variable, tf.placeholder 존재
  
  - tf.Session -> tf.run()으로 실행

- tf.data.Dataset
  - slice 생성 -> Iterator 생성

- tf.layer
  - tf.layers : 함수 tf.Tensor, tf.Operator return
  - shortcut layer : 함수 선언 및 input 넣기 가능
  - 반드시 shape 입력
  
- tf.losses : 여러 종류의 loss 저장

- tf.train.optimizer : optimizer 저장
  - optimizer.minimize(loss) : train function return
  - sess.run(train, loss) -> return이 없어 loss 확인시 같이 입력